# Prompt Enginnering
### **Course Name:** Natural Language Processing **<font color="red">(CSC5051)</font>**




Hello, everyone.
In this tutorial, we'll explore *Prompt Engineering* techniques.

**<font color="blue">What is prompt engineering in AI?</font>**

An AI prompt is a carefully crafted instruction given to an AI model to generate a specific output. These inputs can range from text and images to videos or even music.

Prompt engineering means writing precise instructions that guide AI models like **ChatGPT** to produce specific and useful responses. It involves designing inputs that an AI can easily understand and act upon, ensuring the output is relevant and accurate.

## **The First Step: Mastering the Use of APIs**



### About API Keys
#### DeepSeek API Keys
❗ Note that you can go [Deepseek](https://platform.deepseek.com/api_keys) to get your api key. Alternatively,  youbcan go to [Siliconflow](https://cloud.siliconflow.cn/) for free credit:
- Go to the website [https://platform.deepseek.com/api_keys](https://platform.deepseek.com/api_keys).
- Setup your api key through setting the environment variable os.environ["DEEPSEEK_API_KEY"]
- Remember to update `DEEPSEEK_BASE_URL` to https://api.siliconflow.cn/v1/chat/completions when using API from SiliconFlow.

#### OpenAI API Keys
Note that we provide a key with 100 US dollars, if it is used up you need to buy the Keys yourself (it may cost you a little bit of money), here is how to buy the keys:
- Go to the website [https://eylink.cn/](https://eylink.cn/buy/7).
- Purchase a 12 RMB key (10 US dollars). (10 dollars should be enough.)
- Fill in the `OPENAI_API_KEY` below with the key you purchased.

🔅 To facilitate easier access to OpenAI's model APIs, we make use of a popular framework langchain.

In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-deepseek
!pip install retrying

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.


In [2]:
import os
import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import random
import json
from retrying import retry
import requests


os.environ["DEEPSEEK_API_KEY"] = "sk-632bbeb3bac1464ba38792342d1f3ba2"
os.environ["DEEPSEEK_BASE_URL"] = "https://api.deepseek.com/v1"
deepseek_chat = ChatDeepSeek(model="deepseek-chat", temperature=1)

😊 You can now engage directly with DeepSeek-Chat using our `invoke` function.

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI assistant, please answer user's question."),
        ("user", "{input}")
    ]
)

model = ChatDeepSeek(model="deepseek-chat")

chain = prompt | model

response = chain.invoke({"input": "Hello"})
print(response)

content='Hello! How can I assist you today? 😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 17, 'total_tokens': 28, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 17}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '00bdfa46-b787-4f19-9ad8-7a13c2015893', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--aa8989f8-c576-4a59-887d-326534f5615b-0' usage_metadata={'input_tokens': 17, 'output_tokens': 11, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


To make the output easy to use, we can apply a output parser to the original output!

In [4]:
chain = prompt | model | StrOutputParser()

response = chain.invoke({"input": "Hello"})
print(response)

Hello! How can I assist you today? 😊


### **LLM Setting**

**1. Model Selection**

You are free to choose from several models. Below are the models DeepSeek offers:
- deepseek-chat (currently pointing to deepseek-v3, offering a balance of capability and cost, highly recommended!)
- deepseek-reasoner (currently pointing to deepseek-r1, extremely sophisticated and intelligent)
For detailed information, please visit [DeepSeek's Website](https://api-docs.deepseek.com/zh-cn/news/news250120).

Note that DeepSeek offers a discount when using API in the midnight, details showing [here](https://api-docs.deepseek.com/zh-cn/quick_start/pricing)

Below are the models OpenAI offers:

- gpt-4o-mini (the most cost-effective model, highly recommended!)
- gpt-4o (offers a balance of capability and cost)
- gpt-4-turbo (extremely sophisticated and intelligent)

For detailed information, please visit [OpenAI's Website](https://platform.openai.com/docs/models/gpt-4-turbo-and-gpt-4).

In [5]:
deepseek_reasoner = ChatDeepSeek(model="deepseek-reasoner", temperature=1)
chain = prompt | deepseek_reasoner | StrOutputParser()
chain.invoke({'input': "hello! what\' your name?"}) # 注意reasoner很贵哦

"Hello! I'm DeepSeek, an AI assistant created by DeepSeek Company. I'm here to help you with questions, conversations, and various tasks. It's nice to meet you! 😊 How can I assist you today?"

**2. Temperature**
Controls the randomness of the model's output. Lower values make responses more deterministic and focused, while higher values allow for more creative and diverse outputs. Use low values for factual tasks and high values for creative tasks.

In [6]:
model = ChatDeepSeek(model="deepseek-chat", temperature=0)
chain = prompt | model | StrOutputParser()

chain.invoke( {"input": 'hello! what\' your name?'})

"Hello! I'm an AI assistant created by DeepSeek. I don't have a personal name, but you can think of me as your helpful assistant! 😊 How can I help you today?"

**3. Top P**
This is nucleus sampling, where only tokens from the top probability mass (up to `top_p`) are considered. Lower values encourage more focused responses, while higher values increase the diversity of possible outputs.

In [7]:
model = ChatDeepSeek(model="deepseek-chat", top_p=0.9)
chain = prompt | model | StrOutputParser()

chain.invoke( {"input": 'hello! what\' your name?'})

"Hello! I'm an AI assistant created by DeepSeek. I don't have a personal name, but you can think of me as your helpful AI companion! How can I assist you today? 😊"

**4. Max Length** This limits the total number of tokens the model can generate, helping control response length and prevent irrelevant output.

In [8]:
model = ChatDeepSeek(model="deepseek-chat", max_tokens=5)
chain = prompt | model | StrOutputParser()

chain.invoke( {"input": 'hello! what\' your name?'})

"Hello! I'm an"

### **Prompting Techniques**

**1. Zero-Shot Prompting** Large language models (LLMs) today, such as GPT-3.5 Turbo, GPT-4, and Claude 3, are tuned to follow instructions and are trained on large amounts of data. Large-scale training makes these models capable of performing some tasks in a "zero-shot" manner. Zero-shot prompting means that the prompt used to interact with the model won't contain examples or demonstrations. The zero-shot prompt directly instructs the model to perform a task without any additional examples to steer it.

In [9]:
model = ChatDeepSeek(model="deepseek-chat")

chain = prompt | model | StrOutputParser()

your_prompt = """Classify the text into neutral, negative or positive.
Text: I think the vacation is okay.
Sentiment:"""

chain.invoke({"input": your_prompt})

'neutral'

**2. Few-Shot Prompting** While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

According to [Touvron et al. 2023](https://arxiv.org/pdf/2302.13971.pdf) few shot properties first appeared when models were scaled to a sufficient size ([Kaplan et al., 2020](https://arxiv.org/abs/2001.08361)).

In [10]:
your_prompt = """This is bad! // Negative
This is awesome! // Positive
Wow that movie was rad! // Positive
What a horrible show! //"""
chain.invoke({"input": your_prompt})

'Negative'



**3. Chain-of-Thought Prompting** Introduced in [Wei et al. (2022)](https://arxiv.org/abs/2201.11903), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

![](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fcot.1933d9fe.png&w=1920&q=75)  

In [11]:
your_prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
Let's think step by step."""
chain.invoke({"input": your_prompt})

"Let's go through this step by step.  \n\n1. **Start**: Bought 10 apples.  \n   → Apples = 10  \n\n2. **Gave 2 apples to the neighbor**:  \n   → 10 − 2 = 8 apples left  \n\n3. **Gave 2 apples to the repairman**:  \n   → 8 − 2 = 6 apples left  \n\n4. **Bought 5 more apples**:  \n   → 6 + 5 = 11 apples  \n\n5. **Ate 1 apple**:  \n   → 11 − 1 = 10 apples  \n\n**Final answer**: **10 apples**"

**4. Self-Consistency** Perhaps one of the more advanced techniques out there for prompt engineering is self-consistency. Proposed by [Wang et al. (2022)](https://arxiv.org/abs/2203.11171), self-consistency aims "to replace the naive greedy decoding used in chain-of-thought prompting". The idea is to sample multiple, diverse reasoning paths through few-shot CoT, and use the generations to select the most consistent answer. This helps to boost the performance of CoT prompting on tasks involving arithmetic and commonsense reasoning.

In [12]:
your_prompt = """**Problem: Calculate the total cost if you buy 3 notebooks at $2 each and 2 pens at $1.50 each.**

**Examples:**

1. **Example Problem:** How many apples can you buy with $10 if each costs $2?
   **Solution:**
   - I have $10.
   - Each apple costs $2.
   - $10 / $2 = 5 apples.
   **Answer: You can buy 5 apples.**

2. **Example Problem:** If a train travels 50 miles in an hour, how far will it travel in 4 hours?
   **Solution:**
   - The train travels 50 miles in one hour.
   - In 4 hours, it will travel 50 miles/hour * 4 hours = 200 miles.
   **Answer: The train will travel 200 miles.**

**Your Task:**

- Use the chain of thought to break down the cost calculation.
- Sample multiple reasoning paths.
- Determine the most consistent calculation across different samples.

**Reasoning:**
- Start by identifying the cost of one category of items:
  - 3 notebooks at $2 each = $6.
- Then, calculate the cost for the other category:
  - 2 pens at $1.50 each = $3.
- Add both amounts to find the total cost:
  - $6 (notebooks) + $3 (pens) = $9.

**Consistency Check:**
- Sample several reasoning paths. For example:
  1. Calculate total cost for notebooks first, then pens, and sum.
  2. Calculate total cost for pens first, then notebooks, and sum.
  3. Directly multiply and add the costs of notebooks and pens.
- Compare the answers and select the most frequently occurring result.

**Final Answer:**
- After verifying consistency across samples, conclude with the most consistent answer.
"""
chain.invoke({"input": your_prompt})


'**Step-by-step reasoning:**\n\n1. **Cost of notebooks:**  \n   3 notebooks × $2 each = $6  \n\n2. **Cost of pens:**  \n   2 pens × $1.50 each = $3  \n\n3. **Total cost:**  \n   $6 + $3 = $9  \n\n**Consistency check with alternate reasoning paths:**\n\n- **Path 1:**  \n  Total notebooks cost = 3 × 2 = $6  \n  Total pens cost = 2 × 1.5 = $3  \n  Total = $9  \n\n- **Path 2:**  \n  Total pens cost = 2 × 1.5 = $3  \n  Total notebooks cost = 3 × 2 = $6  \n  Total = $9  \n\n- **Path 3:**  \n  Direct total = (3 × 2) + (2 × 1.5) = 6 + 3 = $9  \n\nAll paths yield the same result: **$9**  \n\n**Final Answer:** The total cost is **$9**.'



**5. Tree of Thoughts (ToT)** For complex tasks that require exploration or strategic lookahead, traditional or simple prompting techniques fall short. [Yao et el. (2023)](https://arxiv.org/abs/2305.10601) and [Long (2023)](https://arxiv.org/abs/2305.08291) recently proposed Tree of Thoughts (ToT), a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem solving with language models.

ToT maintains a tree of thoughts, where thoughts represent coherent language sequences that serve as intermediate steps toward solving a problem. This approach enables an LM to self-evaluate the progress through intermediate thoughts made towards solving a problem through a deliberate reasoning process. The LM's ability to generate and evaluate thoughts is then combined with search algorithms (e.g., breadth-first search and depth-first search) to enable systematic exploration of thoughts with lookahead and backtracking.



![](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2FTOT.3b13bc5e.png&w=3840&q=75)  

In [13]:
your_prompt = """Imagine you are solving a complex logic puzzle where you need to arrange six objects in a specific order based on a set of rules. Each object can only be placed once, and certain objects must be placed before others according to the rules provided below. You need to explore different arrangements systematically to find the correct solution. Follow these steps:_

1. **Step 1: Start by listing possible first moves**: Consider each of the six objects and think through which ones could logically come first based on the rules.
2. **Step 2: Generate multiple intermediate thoughts for the second position**: After selecting a first object, think about which objects can go next while considering the constraints. Explore at least three different possibilities.
3. **Step 3: Evaluate each option**: After placing the first two objects, evaluate whether the current arrangement aligns with the rules. If it does, proceed to explore the third position. If not, backtrack and try another path.
4. **Step 4: Search using Breadth-First Search (BFS)**: Expand on each potential arrangement one step at a time. Use BFS to keep track of multiple arrangements at once, evaluating their adherence to the rules as you go.
5. **Step 5: Look ahead and refine**: After placing three objects, look ahead to the remaining positions and evaluate potential placements. If a placement leads to a conflict, backtrack and explore a different arrangement. Use depth-first search (DFS) if necessary to explore more deeply.
6. **Step 6: Complete the arrangement and find the correct solution**: Continue exploring arrangements, self-evaluating each step, until the correct order is found. Summarize your reasoning process after completing the task."""
chain.invoke({"input": your_prompt})

'Let’s go step-by-step. Since you didn’t provide the specific objects or rules, I’ll create a sample puzzle to demonstrate the reasoning process.  \n\n---\n\n**Example Puzzle Setup**  \nObjects: A, B, C, D, E, F  \n\nRules:  \n1. A must come before B.  \n2. C must come before D.  \n3. E cannot be last.  \n4. B cannot be first.  \n5. D must come before E.  \n6. F must be either first or last.  \n\n---\n\n### **Step 1: List possible first moves**  \nFrom the rules:  \n- B cannot be first (Rule 4)  \n- F must be first or last (Rule 6) → so F could be first.  \n- A, C, D, E could be first? Let’s check constraints:  \n  - If A is first, no problem yet.  \n  - If C is first, no problem yet.  \n  - If D is first, then C must come before D (Rule 2) → conflict unless C is before D, but if D is first, C cannot be before D → so D cannot be first.  \n  - If E is first, then D must come before E (Rule 5) → conflict → E cannot be first.  \n\nSo possible first objects: A, C, F.  \n\n---\n\n### **Step

**6. Retrieval Augmented Generation (RAG)** General-purpose language models can be fine-tuned to achieve several common tasks such as sentiment analysis and named entity recognition. These tasks generally don't require additional background knowledge.

For more complex and knowledge-intensive tasks, it's possible to build a language model-based system that accesses external knowledge sources to complete tasks. This enables more factual consistency, improves reliability of the generated responses, and helps to mitigate the problem of "hallucination".

Meta AI researchers introduced a method called [Retrieval Augmented Generation (RAG)](https://ai.facebook.com/blog/retrieval-augmented-generation-streamlining-the-creation-of-intelligent-natural-language-processing-models/) to address such knowledge-intensive tasks. RAG combines an information retrieval component with a text generator model. RAG can be fine-tuned and its internal knowledge can be modified in an efficient manner and without needing retraining of the entire model.


![](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Frag.c6528d99.png&w=1920&q=75)  

A recommended repository of implementations: (langchain) [https://github.com/langchain-ai/langchain].

In [14]:
your_prompt = """## Instruction: Use the provided retrieval content to answer the question.
### Retrieval Content:
1. The retrieved document discusses the impact of climate change on polar bear populations in the Arctic, detailing factors like ice melting, loss of habitat, and changes in prey availability.

### Question:
What are the primary reasons for the decline in polar bear populations as discussed in the retrieval content, and what measures can be implemented to mitigate this issue?
"""
chain.invoke({"input": your_prompt})

'Based on the retrieval content provided, the primary reasons for the decline in polar bear populations are:\n\n1. **Ice melting** - Reduction of sea ice due to climate change\n2. **Loss of habitat** - Diminishing Arctic ice platforms that polar bears depend on\n3. **Changes in prey availability** - Disruption to the availability of their primary food sources\n\nTo mitigate this issue, while not explicitly detailed in the retrieval content, common conservation measures typically include:\n- Reducing greenhouse gas emissions to address climate change\n- Protecting critical polar bear habitats\n- Implementing sustainable management of Arctic ecosystems\n- Supporting international agreements on climate action and wildlife protection\n- Conducting ongoing research and monitoring of polar bear populations and their ecosystems'

**7. Automatic Reasoning and Tool-use (ART)** Combining CoT prompting and tools in an interleaved manner has shown to be a strong and robust approach to address many tasks with LLMs. These approaches typically require hand-crafting task-specific demonstrations and carefully scripted interleaving of model generations with tool use. Paranjape et al., (2023) propose a new framework that uses a frozen LLM to automatically generate intermediate reasoning steps as a program.

ART works as follows:

- given a new task, it select demonstrations of multi-step reasoning and tool use from a task library
- at test time, it pauses generation whenever external tools are called, and integrate their output before resuming generation

ART encourages the model to generalize from demonstrations to decompose a new task and use tools in appropriate places, in a zero-shot fashion. In addition, ART is extensible as it also enables humans to fix mistakes in the reasoning steps or add new tools by simply updating the task and tool libraries. The process is demonstrated below:

![](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2FART.3b30f615.png&w=1200&q=75)  

## **Task 1: LLMs as a knowledgeable doctor**




The pharmacist licensure exam  is a cornerstone in the pharmacy profession, ensuring that candidates possess the requisite knowledge and skills for safe and effective practice. Its significance lies not only in validating credentials but also in safeguarding public health, enabling professional recognition, and ensuring adherence to legal and regulatory standards.

Advanced models like ChatGPT have significant potential in exam preparation, boasting an extensive knowledge base and the capability to provide in-depth explanations and clarify complex concepts. However, despite the prowess of such large models, if prompts are not designed appropriately, the information retrieved might be inaccurate or incomplete, potentially hindering success in the pharmacist exam.

In [5]:
!wget https://NLP-course-cuhksz.github.io/Assignments/Assignment1/task1/data/1.exam.json

--2025-10-29 14:55:03--  https://nlp-course-cuhksz.github.io/Assignments/Assignment1/task1/data/1.exam.json
Resolving nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86227 (84K) [application/json]
Saving to: ‘1.exam.json’

1.exam.json         100%[===================>]  84.21K  --.-KB/s    in 0.01s   

2025-10-29 14:55:03 (6.59 MB/s) - ‘1.exam.json’ saved [86227/86227]



In [6]:
import json
with open('1.exam.json') as f:
  data = json.load(f)

In [7]:
data[0]

{'question': '27. 根据国家药品监督管理局，公安部，国家卫⽣健康委员会的有关规定，⼜服固体制剂每剂量单位含羟考酮碱不超过5毫克，且不含其他⿇醉药品，精神药品或者药品类易制毒化学品的复⽅制剂列⼊（）。',
 'option': {'A': '含⿇醉药品复⽅制剂的管理',
  'B': '第⼆类精神药品管理',
  'C': '第⼀类精神药品管理',
  'D': '医疗⽤毒性药品管理',
  'E': ''},
 'analysis': '⼜服固体制剂每剂量单位含羟考酮碱不超过5毫克，且不含其他⿇醉药品、精神药品或药品类易制毒化学品的复⽅制剂列⼊第⼆类精神药品管理。',
 'answer': 'B',
 'question_type': '最佳选择题',
 'source': '2021年执业药师职业资格考试《药事管理与法规》'}

In [8]:
your_prompt = """请回答下面的多选题，请直接正确答案选项，不要输出其他内容。
{question}
{options}"""

def get_query(da):
  da['options'] = '\n'.join([f"{k}:{v}" for k,v in da['option'].items()])
  return your_prompt.format_map(da)

get_query(data[0])

'请回答下面的多选题，请直接正确答案选项，不要输出其他内容。\n27. 根据国家药品监督管理局，公安部，国家卫⽣健康委员会的有关规定，⼜服固体制剂每剂量单位含羟考酮碱不超过5毫克，且不含其他⿇醉药品，精神药品或者药品类易制毒化学品的复⽅制剂列⼊（）。\nA:含⿇醉药品复⽅制剂的管理\nB:第⼆类精神药品管理\nC:第⼀类精神药品管理\nD:医疗⽤毒性药品管理\nE:'

In [9]:
chain.invoke(get_query(data[0]))
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [10]:
# ===============================
# Case Study: 比较 CoT 和 ART 方法
# 找出 CoT 错误但 ART 正确的题目
# ===============================

import re
import time
from tqdm import tqdm
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 初始化模型
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant, please answer user's question."),
    ("user", "{input}")
])
model = ChatDeepSeek(model="deepseek-chat", temperature=0)
chain = prompt | model | StrOutputParser()

# ===============================
# 1. CoT 方法评估（记录详细结果）
# ===============================

prompt_cot = """你是资深药剂师，请分析以下考题：

题目：{question}

选项：
{options}

让我们一步一步地思考并分析每个选项。最后输出答案（格式：答案：A 或 答案：BC）"""

def get_query_cot(da):
    """构建 CoT 查询"""
    options_text = '\n'.join([f"{k}: {v}" for k, v in da['option'].items() if v])
    return prompt_cot.format(question=da['question'], options=options_text)

def get_ans(ans_text):
    """提取答案字母"""
    answer_match = re.search(r'答案[：:]\s*([A-E]+)', ans_text)
    if answer_match:
        return answer_match.group(1)

    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans_text)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

print("="*70)
print("步骤 1: 运行 CoT 方法")
print("="*70)

cot_results = []
for i, da in enumerate(tqdm(data[:100], desc="CoT 测试")):
    query = get_query_cot(da)
    response = chain.invoke({"input": query})

    predicted_ans = get_ans(response)
    is_correct = (predicted_ans == da['answer'])

    cot_results.append({
        'index': i,
        'question': da['question'],
        'options': da['option'],
        'correct_answer': da['answer'],
        'predicted_answer': predicted_ans,
        'full_response': response,
        'is_correct': is_correct
    })

cot_correct = sum(1 for r in cot_results if r['is_correct'])
print(f"\nCoT 准确率: {cot_correct}/{len(cot_results)} = {cot_correct/len(cot_results):.2%}")

# ===============================
# 2. ART 方法评估（记录详细结果）
# ===============================

def build_knowledge_base(data_subset):
    """从 analysis 构建知识库"""
    kb = {}
    for item in data_subset:
        if item.get('analysis'):
            words = re.findall(r'[\u4e00-\u9fa5]{2,10}', item['question'])
            for word in words[:8]:
                if word not in kb:
                    kb[word] = []
                if item['analysis'] not in kb[word]:
                    kb[word].append(item['analysis'])
    return kb

def tool_search_knowledge(keyword, kb):
    """外部工具：知识库查询"""
    results = []
    for key, analyses in kb.items():
        if keyword in key or key in keyword or keyword[:2] in key:
            results.extend(analyses[:1])
            if len(results) >= 2:
                break

    if results:
        return "【工具返回】\n" + "\n".join(set(results[:2]))
    else:
        return "【工具返回】未找到相关信息"

prompt_art_stage1 = """你是智能药师 Agent。快速判断是否需要查询外部知识库：

题目：{question}

如需查询，输出：需要工具：<关键词>
如无需，输出：无需工具

只输出一行："""

prompt_art_stage2 = """你是专业药师。请结合外部工具返回的信息作答：

题目：{question}
选项：{options}

{tool_result}

请推理并在最后一行输出：答案：<字母>"""

print("\n" + "="*70)
print("步骤 2: 运行 ART 方法（2-Fold Cross Validation）")
print("="*70)

art_results = [None] * 100  # 预先分配列表

# Fold 1: 前50建库 → 后50测试
kb1 = build_knowledge_base(data[:50])
print(f"\nFold 1: 知识库包含 {len(kb1)} 个关键词")

for i in tqdm(range(50, 100), desc="Fold 1 测试"):
    da = data[i]
    question = da['question']
    options = '\n'.join([f"{k}: {v}" for k, v in da['option'].items() if v])

    # Stage 1: 工具决策
    query1 = prompt_art_stage1.format(question=question)
    response1 = chain.invoke({"input": query1})

    tool_result_text = ""
    if "需要工具" in response1:
        match = re.search(r'需要工具[：:]\s*(.+)', response1)
        if match:
            keyword = match.group(1).strip()[:15]
            tool_result_text = tool_search_knowledge(keyword, kb1)
    else:
        tool_result_text = "（Agent 决定：无需外部工具）"

    # Stage 2: 推理和作答
    query2 = prompt_art_stage2.format(
        question=question,
        options=options,
        tool_result=tool_result_text
    )
    response2 = chain.invoke({"input": query2})

    predicted_ans = get_ans(response2)
    is_correct = (predicted_ans == da['answer'])

    art_results[i] = {
        'index': i,
        'question': da['question'],
        'options': da['option'],
        'correct_answer': da['answer'],
        'predicted_answer': predicted_ans,
        'tool_decision': response1,
        'tool_result': tool_result_text,
        'full_response': response2,
        'is_correct': is_correct
    }

# Fold 2: 后50建库 → 前50测试
kb2 = build_knowledge_base(data[50:100])
print(f"\nFold 2: 知识库包含 {len(kb2)} 个关键词")

for i in tqdm(range(50), desc="Fold 2 测试"):
    da = data[i]
    question = da['question']
    options = '\n'.join([f"{k}: {v}" for k, v in da['option'].items() if v])

    # Stage 1: 工具决策
    query1 = prompt_art_stage1.format(question=question)
    response1 = chain.invoke({"input": query1})

    tool_result_text = ""
    if "需要工具" in response1:
        match = re.search(r'需要工具[：:]\s*(.+)', response1)
        if match:
            keyword = match.group(1).strip()[:15]
            tool_result_text = tool_search_knowledge(keyword, kb2)
    else:
        tool_result_text = "（Agent 决定：无需外部工具）"

    # Stage 2: 推理和作答
    query2 = prompt_art_stage2.format(
        question=question,
        options=options,
        tool_result=tool_result_text
    )
    response2 = chain.invoke({"input": query2})

    predicted_ans = get_ans(response2)
    is_correct = (predicted_ans == da['answer'])

    art_results[i] = {
        'index': i,
        'question': da['question'],
        'options': da['option'],
        'correct_answer': da['answer'],
        'predicted_answer': predicted_ans,
        'tool_decision': response1,
        'tool_result': tool_result_text,
        'full_response': response2,
        'is_correct': is_correct
    }

art_correct = sum(1 for r in art_results if r['is_correct'])
print(f"\nART 准确率: {art_correct}/{len(art_results)} = {art_correct/len(art_results):.2%}")

# ===============================
# 3. 对比分析：找出 CoT 错但 ART 对的题目
# ===============================

print("\n" + "="*70)
print("步骤 3: 对比分析")
print("="*70)

cot_wrong_art_right = []

for i in range(100):
    cot_result = cot_results[i]
    art_result = art_results[i]

    if not cot_result['is_correct'] and art_result['is_correct']:
        cot_wrong_art_right.append({
            'index': i,
            'cot': cot_result,
            'art': art_result
        })

print(f"\nCoT 错误但 ART 正确的题目数量: {len(cot_wrong_art_right)}")
print(f"CoT 正确: {cot_correct}/100")
print(f"ART 正确: {art_correct}/100")
print(f"改进题目数: {len(cot_wrong_art_right)}")

# ===============================
# 4. 展示详细结果
# ===============================

print("\n" + "="*70)
print("步骤 4: 展示 CoT 错误但 ART 正确的题目详情")
print("="*70)

for idx, case in enumerate(cot_wrong_art_right, 1):
    print(f"\n{'='*70}")
    print(f"案例 {idx} - 题目编号: {case['index']}")
    print(f"{'='*70}")

    cot = case['cot']
    art = case['art']

    print(f"\n【题目】")
    print(cot['question'])

    print(f"\n【选项】")
    for k, v in cot['options'].items():
        if v:
            print(f"{k}: {v}")

    print(f"\n【正确答案】 {cot['correct_answer']}")

    print(f"\n" + "-"*70)
    print(f"【CoT 方法】")
    print(f"-"*70)
    print(f"预测答案: {cot['predicted_answer']}")
    print(f"判定: ❌ 错误")
    print(f"\n完整输出:")
    print(cot['full_response'])

    print(f"\n" + "-"*70)
    print(f"【ART 方法】")
    print(f"-"*70)
    print(f"预测答案: {art['predicted_answer']}")
    print(f"判定: ✅ 正确")

    print(f"\n工具决策:")
    print(art['tool_decision'])

    print(f"\n工具返回:")
    print(art['tool_result'])

    print(f"\n完整输出:")
    print(art['full_response'])

    print(f"\n{'='*70}\n")

print(f"\n✅ 分析完成！共找到 {len(cot_wrong_art_right)} 道 CoT 错误但 ART 正确的题目。")

步骤 1: 运行 CoT 方法


CoT 测试: 100%|██████████| 100/100 [24:02<00:00, 14.43s/it]



CoT 准确率: 87/100 = 87.00%

步骤 2: 运行 ART 方法（2-Fold Cross Validation）

Fold 1: 知识库包含 245 个关键词


Fold 1 测试: 100%|██████████| 50/50 [07:12<00:00,  8.66s/it]



Fold 2: 知识库包含 190 个关键词


Fold 2 测试: 100%|██████████| 50/50 [07:05<00:00,  8.51s/it]


ART 准确率: 88/100 = 88.00%

步骤 3: 对比分析

CoT 错误但 ART 正确的题目数量: 5
CoT 正确: 87/100
ART 正确: 88/100
改进题目数: 5

步骤 4: 展示 CoT 错误但 ART 正确的题目详情

案例 1 - 题目编号: 4

【题目】
19. 根据药品管理法律法规及相关⽂件的规定，药品零售企业可以开架⾃选销售的药品是（）。

【选项】
A: 复⽅⽢草⽚
B: 复⽅⾦银花颗粒
C: 复⽅地芬诺酯⽚
D: 复⽅枇杷喷托维林颗粒

【正确答案】 B

----------------------------------------------------------------------
【CoT 方法】
----------------------------------------------------------------------
预测答案: BD
判定: ❌ 错误

完整输出:
好的，我们先一步步分析。  

---

**1. 题目考点**  
题目问的是“药品零售企业可以开架自选销售的药品”，这涉及到药品分类管理规定。  
根据中国《药品经营质量管理规范》及相关规定：  
- **非处方药（OTC）** 可以开架自选销售，分为甲类OTC（须药师指导）和乙类OTC（安全性更高，可超市等销售）。  
- **处方药** 必须凭处方销售，不得开架自选。  
- 某些特殊药品（如含麻制剂、精神药品等）即使是非处方药，也可能有专门限制，不能开架。  

---

**2. 分析每个选项**  

**A: 复方甘草片**  
- 含阿片粉，属于“含麻醉药品口服复方制剂”，必须凭处方销售，且不得开架自选，有严格登记要求。  
- ❌ 不能开架自选。  

**B: 复方金银花颗粒**  
- 常见中成药，一般是 OTC（乙类非处方药），安全性高。  
- ✅ 可以开架自选。  

**C: 复方地芬诺酯片**  
- 地芬诺酯属于麻醉药品（止泻药），被列为“必须凭处方销售的药品”，且是特殊管理药品，严禁开架销售。  
- ❌ 不能开架自选。  

**D: 复方枇杷喷托维林颗粒**  
- 喷托维林是镇咳药，复方枇杷喷托维林颗粒属于 OTC（常见为甲类或乙类）。  
- 查药品数据库：喷托维林

In [13]:
# ===============================
# 辅助函数：查看单个题目的详细对比
# ===============================

def show_comparison(question_index):
    """显示指定题目的 CoT 和 ART 详细对比"""

    if question_index < 0 or question_index >= 100:
        print("❌ 题目索引超出范围（0-99）")
        return

    cot = cot_results[question_index]
    art = art_results[question_index]

    print(f"{'='*70}")
    print(f"题目编号: {question_index}")
    print(f"{'='*70}")

    print(f"\n【题目】")
    print(cot['question'])

    print(f"\n【选项】")
    for k, v in cot['options'].items():
        if v:
            print(f"{k}: {v}")

    print(f"\n【正确答案】 {cot['correct_answer']}")

    print(f"\n" + "-"*70)
    print(f"【CoT 方法】")
    print(f"-"*70)
    print(f"预测答案: {cot['predicted_answer']}")
    print(f"判定: {'✅ 正确' if cot['is_correct'] else '❌ 错误'}")
    print(f"\n完整输出:")
    print(cot['full_response'])

    print(f"\n" + "-"*70)
    print(f"【ART 方法】")
    print(f"-"*70)
    print(f"预测答案: {art['predicted_answer']}")
    print(f"判定: {'✅ 正确' if art['is_correct'] else '❌ 错误'}")

    print(f"\n工具决策:")
    print(art['tool_decision'])

    print(f"\n工具返回:")
    print(art['tool_result'])

    print(f"\n完整输出:")
    print(art['full_response'])

    print(f"\n{'='*70}\n")

# 【CoT 错误但 ART 正确的题目索引】
# [4, 20, 43, 70, 86]

# 【CoT 正确但 ART 错误的题目索引】
# [11, 17, 25, 67]
# 使用示例：
show_comparison(4)  # 查看第 0 道题
show_comparison(20)
show_comparison(43)
show_comparison(70)
show_comparison(86)
print("***********************************")
show_comparison(11)
show_comparison(17)
show_comparison(25)
show_comparison(67)
# show_comparison(27) # 查看第 27 道题

题目编号: 4

【题目】
19. 根据药品管理法律法规及相关⽂件的规定，药品零售企业可以开架⾃选销售的药品是（）。

【选项】
A: 复⽅⽢草⽚
B: 复⽅⾦银花颗粒
C: 复⽅地芬诺酯⽚
D: 复⽅枇杷喷托维林颗粒

【正确答案】 B

----------------------------------------------------------------------
【CoT 方法】
----------------------------------------------------------------------
预测答案: BD
判定: ❌ 错误

完整输出:
好的，我们先一步步分析。  

---

**1. 题目考点**  
题目问的是“药品零售企业可以开架自选销售的药品”，这涉及到药品分类管理规定。  
根据中国《药品经营质量管理规范》及相关规定：  
- **非处方药（OTC）** 可以开架自选销售，分为甲类OTC（须药师指导）和乙类OTC（安全性更高，可超市等销售）。  
- **处方药** 必须凭处方销售，不得开架自选。  
- 某些特殊药品（如含麻制剂、精神药品等）即使是非处方药，也可能有专门限制，不能开架。  

---

**2. 分析每个选项**  

**A: 复方甘草片**  
- 含阿片粉，属于“含麻醉药品口服复方制剂”，必须凭处方销售，且不得开架自选，有严格登记要求。  
- ❌ 不能开架自选。  

**B: 复方金银花颗粒**  
- 常见中成药，一般是 OTC（乙类非处方药），安全性高。  
- ✅ 可以开架自选。  

**C: 复方地芬诺酯片**  
- 地芬诺酯属于麻醉药品（止泻药），被列为“必须凭处方销售的药品”，且是特殊管理药品，严禁开架销售。  
- ❌ 不能开架自选。  

**D: 复方枇杷喷托维林颗粒**  
- 喷托维林是镇咳药，复方枇杷喷托维林颗粒属于 OTC（常见为甲类或乙类）。  
- 查药品数据库：喷托维林单方是 OTC 乙类，复方制剂一般也是 OTC，但需确认是否在禁止开架的“含特殊药品复方制剂”名单中。  
- 含喷托维林的复方制剂，不属于“含麻制剂”或“含精神药品复方制剂”限制开架的名单（限制开架的主要是含麻黄碱类、可待因、阿片、地芬诺酯等）。  
- 因此，此药通常按 O

In [11]:


# ===============================
# 统计分析和可视化
# ===============================

print("="*70)
print("方法对比统计分析")
print("="*70)

# 统计四种情况
both_correct = sum(1 for i in range(100) if cot_results[i]['is_correct'] and art_results[i]['is_correct'])
both_wrong = sum(1 for i in range(100) if not cot_results[i]['is_correct'] and not art_results[i]['is_correct'])
cot_only = sum(1 for i in range(100) if cot_results[i]['is_correct'] and not art_results[i]['is_correct'])
art_only = sum(1 for i in range(100) if not cot_results[i]['is_correct'] and art_results[i]['is_correct'])

print(f"\n【准确率对比】")
print(f"CoT 准确率:  {sum(r['is_correct'] for r in cot_results)}/100 = {sum(r['is_correct'] for r in cot_results)}%")
print(f"ART 准确率:  {sum(r['is_correct'] for r in art_results)}/100 = {sum(r['is_correct'] for r in art_results)}%")
print(f"提升幅度:    +{sum(r['is_correct'] for r in art_results) - sum(r['is_correct'] for r in cot_results)}%")

print(f"\n【详细分类】")
print(f"两种方法都正确:         {both_correct} 题")
print(f"两种方法都错误:         {both_wrong} 题")
print(f"仅 CoT 正确（ART 错）:  {cot_only} 题")
print(f"仅 ART 正确（CoT 错）:  {art_only} 题")

print(f"\n【ART 的改进分析】")
if art_only > 0:
    print(f"✅ ART 成功纠正了 CoT 的 {art_only} 个错误")
if cot_only > 0:
    print(f"⚠️  但 ART 也引入了 {cot_only} 个新错误")

net_improvement = art_only - cot_only
if net_improvement > 0:
    print(f"📈 净改进: +{net_improvement} 题")
elif net_improvement < 0:
    print(f"📉 净退步: {net_improvement} 题")
else:
    print(f"➡️  整体持平")

# 导出 CoT 错 ART 对的题目索引
print(f"\n【CoT 错误但 ART 正确的题目索引】")
cot_wrong_art_right_indices = [i for i in range(100)
                               if not cot_results[i]['is_correct'] and art_results[i]['is_correct']]
print(cot_wrong_art_right_indices)

# 导出 CoT 对 ART 错的题目索引
print(f"\n【CoT 正确但 ART 错误的题目索引】")
cot_right_art_wrong_indices = [i for i in range(100)
                               if cot_results[i]['is_correct'] and not art_results[i]['is_correct']]
print(cot_right_art_wrong_indices)

print("\n" + "="*70)

方法对比统计分析

【准确率对比】
CoT 准确率:  87/100 = 87%
ART 准确率:  88/100 = 88%
提升幅度:    +1%

【详细分类】
两种方法都正确:         83 题
两种方法都错误:         8 题
仅 CoT 正确（ART 错）:  4 题
仅 ART 正确（CoT 错）:  5 题

【ART 的改进分析】
✅ ART 成功纠正了 CoT 的 5 个错误
⚠️  但 ART 也引入了 4 个新错误
📈 净改进: +1 题

【CoT 错误但 ART 正确的题目索引】
[4, 20, 43, 70, 86]

【CoT 正确但 ART 错误的题目索引】
[11, 17, 25, 67]



In [ ]:
# ===分割=== 上面研究 case study

In [12]:
# ===============================
# 方案 4：Agent-based — Simplified ART
# ===============================

# 导入必要的库（如果之前已经导入过就不用重复）
import re
from tqdm import tqdm

# ART Prompt
prompt_art = """你是智能药师 Agent，具备推理和工具调用能力。

题目：{question}

选项：
{options}

可用工具：
- search_drug_database(药物名): 查询药物信息
- search_regulation(法规关键词): 查询药事法规
- search_clinical_guide(疾病名): 查询临床用药指南

请按以下格式回答：
1. 【分析】简要分析题目类型和考点
2. 【工具决策】说明是否需要调用工具，如需要，写出：需要工具：<工具名(参数)>；如不需要，写：无需工具
3. 【推理】基于你的知识进行推理
4. 【答案】答案：<字母>

现在开始作答："""

# 答案提取函数（确保已定义）
def get_ans(ans_text):
    """提取模型输出中的答案字母"""
    # 优先匹配 "答案：" 后面的字母
    answer_match = re.search(r'答案[：:]\s*([A-E]+)', ans_text)
    if answer_match:
        return answer_match.group(1)

    # 如果没有找到，使用通用逻辑
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans_text)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

# 评估函数
def evaluate_prompt(prompt_template, data, model_chain):
    """计算模型在指定 prompt 下的准确率"""
    correct_num = 0
    total_num = 0
    for da in tqdm(data[:100]):
        da['options'] = '\n'.join([f"{k}: {v}" for k, v in da['option'].items() if v])
        query = prompt_template.format(question=da['question'], options=da['options'])
        response = model_chain.invoke({"input": query})
        da['deepseek_ans'] = response
        if get_ans(response) == da['answer']:
            correct_num += 1
        total_num += 1
    acc = correct_num / total_num
    print(f"模型准确率: {acc:.2%}")
    return acc

# 运行测试
print("\n=== 方案 4：Simplified Agent-based (ART) ===")
acc_art = evaluate_prompt(prompt_art, data, chain)


=== 方案 4：Simplified Agent-based (ART) ===


100%|██████████| 100/100 [15:25<00:00,  9.25s/it]

模型准确率: 87.00%


In [12]:
# ===============================
# Agent-based ART with 2-Fold Cross Validation
# ===============================

import re
from tqdm import tqdm
import time

# 1) 构建知识库函数
def build_knowledge_base(data_subset):
    """从 analysis 构建知识库"""
    kb = {}
    for item in data_subset:
        if item.get('analysis'):
            # 提取关键词（中文药物名、疾病名等）
            words = re.findall(r'[\u4e00-\u9fa5]{2,10}', item['question'])
            for word in words[:8]:  # 每题最多取8个关键词
                if word not in kb:
                    kb[word] = []
                if item['analysis'] not in kb[word]:
                    kb[word].append(item['analysis'])
    return kb

# 2) 工具函数
def tool_search_knowledge(keyword, kb):
    """外部工具：知识库查询"""
    results = []
    for key, analyses in kb.items():
        if keyword in key or key in keyword or keyword[:2] in key:
            results.extend(analyses[:1])
            if len(results) >= 2:
                break

    if results:
        return "【工具返回】\n" + "\n".join(set(results[:2]))
    else:
        return "【工具返回】未找到相关信息"

# 3) 两阶段 ART Prompt
prompt_art_stage1 = """你是智能药师 Agent。快速判断是否需要查询外部知识库：

题目：{question}

如需查询，输出：需要工具：<关键词>
如无需，输出：无需工具

只输出一行："""

prompt_art_stage2 = """你是专业药师。请结合外部工具返回的信息作答：

题目：{question}
选项：{options}

{tool_result}

请推理并在最后一行输出：答案：<字母>"""

# 4) 答案提取函数（确保已定义）
def get_ans(ans_text):
    """提取模型输出中的答案字母"""
    answer_match = re.search(r'答案[：:]\s*([A-E]+)', ans_text)
    if answer_match:
        return answer_match.group(1)

    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans_text)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

# 5) 单次 ART 评估（一个 fold）
def evaluate_art_single_fold(train_data, test_data, model_chain, fold_name):
    """评估单个 fold"""

    # 构建知识库
    kb = build_knowledge_base(train_data)
    print(f"\n{fold_name}: 知识库构建完成，包含 {len(kb)} 个关键词")

    correct_num = 0
    tool_used_count = 0
    tool_helpful_count = 0
    total_time = 0
    reasoning_lengths = []

    for da in tqdm(test_data, desc=f"{fold_name} 测试中"):
        question = da['question']
        options = '\n'.join([f"{k}: {v}" for k, v in da['option'].items() if v])

        start_time = time.time()

        # === Stage 1: 工具决策 ===
        query1 = prompt_art_stage1.format(question=question)
        response1 = model_chain.invoke({"input": query1})

        tool_used = False
        tool_result_text = ""

        if "需要工具" in response1:
            tool_used = True
            tool_used_count += 1

            match = re.search(r'需要工具[：:]\s*(.+)', response1)
            if match:
                keyword = match.group(1).strip()[:15]
                tool_result_text = tool_search_knowledge(keyword, kb)
        else:
            tool_result_text = "（Agent 决定：无需外部工具）"

        # === Stage 2: 推理和作答 ===
        query2 = prompt_art_stage2.format(
            question=question,
            options=options,
            tool_result=tool_result_text
        )
        response2 = model_chain.invoke({"input": query2})

        end_time = time.time()
        elapsed = end_time - start_time
        total_time += elapsed

        # 提取答案
        predicted = get_ans(response2)
        is_correct = (predicted == da['answer'])

        if is_correct:
            correct_num += 1
            if tool_used and "【工具返回】" in tool_result_text and "未找到" not in tool_result_text:
                tool_helpful_count += 1

        reasoning_lengths.append(len(response2))

    total = len(test_data)

    return {
        'correct': correct_num,
        'total': total,
        'accuracy': correct_num / total,
        'avg_time': total_time / total,
        'total_time': total_time,
        'tool_used': tool_used_count,
        'tool_usage_rate': tool_used_count / total,
        'tool_helpful': tool_helpful_count,
        'avg_reasoning_length': sum(reasoning_lengths) / len(reasoning_lengths)
    }

# 6) 两折交叉验证主函数
def evaluate_agent_art_2fold(data, model_chain):
    """两折交叉验证评估 Agent-based ART"""

    print("\n" + "="*70)
    print("Agent-based ART: 两折交叉验证")
    print("="*70)

    # 划分数据
    fold1_train = data[:50]   # 前50题作训练集
    fold1_test = data[50:100]  # 后50题作测试集

    fold2_train = data[50:100] # 后50题作训练集
    fold2_test = data[:50]     # 前50题作测试集

    # Fold 1: 前50建库 → 后50测试
    print("\n【Fold 1】前50题建库 → 后50题测试")
    fold1_results = evaluate_art_single_fold(fold1_train, fold1_test, model_chain, "Fold 1")

    # Fold 2: 后50建库 → 前50测试
    print("\n【Fold 2】后50题建库 → 前50题测试")
    fold2_results = evaluate_art_single_fold(fold2_train, fold2_test, model_chain, "Fold 2")

    # 汇总结果
    total_correct = fold1_results['correct'] + fold2_results['correct']
    total_samples = fold1_results['total'] + fold2_results['total']
    avg_accuracy = (fold1_results['accuracy'] + fold2_results['accuracy']) / 2
    avg_time = (fold1_results['avg_time'] + fold2_results['avg_time']) / 2
    total_time = fold1_results['total_time'] + fold2_results['total_time']
    avg_tool_usage = (fold1_results['tool_usage_rate'] + fold2_results['tool_usage_rate']) / 2
    total_tool_helpful = fold1_results['tool_helpful'] + fold2_results['tool_helpful']
    avg_reasoning_length = (fold1_results['avg_reasoning_length'] + fold2_results['avg_reasoning_length']) / 2

    # 输出汇总结果
    print("\n" + "="*70)
    print("Agent-based ART 两折交叉验证汇总")
    print("="*70)
    print(f"Fold 1 准确率:            {fold1_results['accuracy']:.2%} ({fold1_results['correct']}/{fold1_results['total']})")
    print(f"Fold 2 准确率:            {fold2_results['accuracy']:.2%} ({fold2_results['correct']}/{fold2_results['total']})")
    print(f"-" * 70)
    print(f"平均准确率 (Accuracy):    {avg_accuracy:.2%}")
    print(f"总体准确率:               {total_correct}/{total_samples} = {total_correct/total_samples:.2%}")
    print(f"平均耗时 (Avg Time):      {avg_time:.2f}秒/题")
    print(f"总耗时 (Total Time):      {total_time/60:.1f}分钟")
    print(f"平均工具使用率:           {avg_tool_usage:.1%}")
    print(f"工具有效贡献总次数:       {total_tool_helpful}次")
    print(f"平均推理长度:             {avg_reasoning_length:.0f}字符")
    print("="*70 + "\n")

    return {
        'method': 'Agent-based (ART)',
        'accuracy': avg_accuracy,
        'avg_time': avg_time,
        'total_time': total_time,
        'tool_usage_rate': avg_tool_usage,
        'tool_helpful': total_tool_helpful,
        'avg_reasoning_length': avg_reasoning_length,
        'fold1': fold1_results,
        'fold2': fold2_results
    }

# 7) 运行两折交叉验证
art_metrics = evaluate_agent_art_2fold(data, chain)


Agent-based ART: 两折交叉验证

【Fold 1】前50题建库 → 后50题测试

Fold 1: 知识库构建完成，包含 245 个关键词


Fold 1 测试中: 100%|██████████| 50/50 [07:57<00:00,  9.56s/it]



【Fold 2】后50题建库 → 前50题测试

Fold 2: 知识库构建完成，包含 190 个关键词


Fold 2 测试中: 100%|██████████| 50/50 [07:51<00:00,  9.44s/it]


Agent-based ART 两折交叉验证汇总
Fold 1 准确率:            94.00% (47/50)
Fold 2 准确率:            86.00% (43/50)
----------------------------------------------------------------------
平均准确率 (Accuracy):    90.00%
总体准确率:               90/100 = 90.00%
平均耗时 (Avg Time):      9.50秒/题
总耗时 (Total Time):      15.8分钟
平均工具使用率:           98.0%
工具有效贡献总次数:       34次
平均推理长度:             247字符



In [16]:
# ===============================
# Few-shot Prompt
# ===============================

from tqdm import tqdm
import re

# 1️⃣ 从前 3 题构造 few-shot 示例
few_shot_examples = ""
for i in range(3):
    q = data[i]['question']
    opts = '\n'.join([f"{k}: {v}" for k, v in data[i]['option'].items() if v])
    ans = data[i]['answer']
    few_shot_examples += f"【示例 {i+1}】\n题目：{q}\n{opts}\n正确答案：{ans}\n\n"

# 2️⃣ 定义 few-shot prompt 模板
prompt_fewshot = f"""以下是几道药师执照考试的示例题及其正确答案。请学习示例的答题风格，并以相同格式回答最后一道题。

{few_shot_examples}

请模仿上述示例的答题方式，回答以下正式考试题。
只输出正确答案选项字母（如：A 或 BC 或 ABD），不要输出任何其他内容。

题目：{{question}}
选项：
{{options}}"""

# 3️⃣ 工具函数
def get_query(da, prompt_template):
    """将题目和选项填入指定 prompt 模板"""
    options_text = '\n'.join([f"{k}: {v}" for k, v in da['option'].items() if v])
    return prompt_template.format(question=da['question'], options=options_text)

def get_ans(ans_text):
    """提取模型输出中的答案字母"""
    match = re.search(r'答案[：:]\s*([A-E]+)', ans_text)
    if match:
        return match.group(1)
    match = re.findall(r'\b[A-E]{1,5}\b', ans_text.upper())
    return match[-1] if match else ''

def evaluate_prompt(prompt_template, data, model_chain, start_index=3):
    """计算模型在指定 prompt 下的准确率"""
    correct_num = 0
    total_num = 0
    for da in tqdm(data[start_index:]):  # 从第 4 题开始测试（避免示例题）
        query = get_query(da, prompt_template)
        # ✅ 取出 AIMessage 的 content
        response = model_chain.invoke({"input": query})
        if hasattr(response, "content"):
            da['deepseek_ans'] = response.content
        else:
            da['deepseek_ans'] = str(response)
        if get_ans(da['deepseek_ans']) == da['answer']:
            correct_num += 1
        total_num += 1
    acc = correct_num / total_num if total_num > 0 else 0
    print(f"Few-shot 模型准确率: {correct_num}/{total_num} = {acc:.2%}")
    return acc

# 4️⃣ 运行 Few-shot 实验
print("\n=== 方案 4：Few-shot Prompt ===")
acc_fewshot = evaluate_prompt(prompt_fewshot, data, chain, start_index=3)


=== 方案 4：Few-shot Prompt ===


100%|██████████| 97/97 [02:08<00:00,  1.33s/it]

Few-shot 模型准确率: 77/97 = 79.38%


In [35]:
# ===============================
# 确保 chain 配置正确
# ===============================
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant, please answer user's question."),
    ("user", "{input}")
])

model = ChatDeepSeek(model="deepseek-chat", temperature=0)  # 设置 temperature=0 保证可复现
chain = prompt | model | StrOutputParser()

# ===============================
# 定义三种 Prompt 模板
# ===============================

# 方案 1：Zero-shot Prompt
prompt_zero = """请回答下列药师执照考试题：

{question}

{options}

只输出所有正确选项的字母，紧凑写在一行（如：A 或 BC 或 ABD）。不要输出任何多余文字。"""

# 方案 2：CoT Prompt
prompt_cot = """你是资深药剂师，请分析以下考题：

题目：{question}

选项：
{options}

让我们一步一步地思考并分析每个选项。最后输出答案（格式：答案：A 或 答案：BC）"""

# 方案 3：Role-based Prompt
prompt_role = """你是一位拥有15年临床经验的资深执业药师，曾多次参与执业药师考试命题和阅卷工作，对药学知识和考试重点非常熟悉。

现在请运用你的专业知识回答以下考题：

题目：{question}

选项：
{options}

请直接输出正确答案（格式：A 或 BC 或 ABD）："""

# ===============================
# 工具函数
# ===============================
import re
from tqdm import tqdm

def get_query(da, prompt_template):
    """将题目和选项填入指定 prompt 模板"""
    options_text = '\n'.join([f"{k}: {v}" for k, v in da['option'].items() if v])
    return prompt_template.format(question=da['question'], options=options_text)

def get_ans(ans_text):
    """提取模型输出中的答案字母（优化版，支持 CoT 格式）"""
    # 优先匹配 "答案：" 后面的字母
    answer_match = re.search(r'答案[：:]\s*([A-E]+)', ans_text)
    if answer_match:
        return answer_match.group(1)

    # 如果没有找到，使用通用逻辑
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans_text)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

def evaluate_prompt(prompt_template, data, model_chain):
    """计算模型在指定 prompt 下的准确率"""
    correct_num = 0
    total_num = 0
    for da in tqdm(data[:100]):
        query = get_query(da, prompt_template)
        da['deepseek_ans'] = model_chain.invoke({"input": query})  # 注意：使用字典格式
        if get_ans(da['deepseek_ans']) == da['answer']:
            correct_num += 1
        total_num += 1
    acc = correct_num / total_num
    print(f"模型准确率: {acc:.2%}")
    return acc

# ===============================
# 分别运行三种方案
# ===============================
print("\n=== 方案 1：Zero-Shot ===")
acc_zero = evaluate_prompt(prompt_zero, data, chain)

print("\n=== 方案 2：Chain-of-Thought ===")
acc_cot = evaluate_prompt(prompt_cot, data, chain)

print("\n=== 方案 3：Role-Based ===")
acc_role = evaluate_prompt(prompt_role, data, chain)

# ===============================
# 结果对比
# ===============================
print("\n" + "="*60)
print("结果对比")
print("="*60)
print(f"{'方法':<20} {'准确率':>10} {'相对提升':>15}")
print("-"*60)
print(f"{'Zero-Shot':<20} {acc_zero:>9.2%} {'+0.0%':>15}")
print(f"{'Chain-of-Thought':<20} {acc_cot:>9.2%} {f'+{(acc_cot-acc_zero)*100:.1f}%':>15}")
print(f"{'Role-Based':<20} {acc_role:>9.2%} {f'+{(acc_role-acc_zero)*100:.1f}%':>15}")
print("="*60)

best_method = max([('Zero-Shot', acc_zero), ('CoT', acc_cot), ('Role-Based', acc_role)],
                  key=lambda x: x[1])
print(f"🏆 最佳方法: {best_method[0]} ({best_method[1]:.2%})")


=== 方案 1：Zero-Shot ===


100%|██████████| 100/100 [02:18<00:00,  1.38s/it]


模型准确率: 73.00%

=== 方案 2：Chain-of-Thought ===


100%|██████████| 100/100 [24:02<00:00, 14.43s/it]


模型准确率: 85.00%

=== 方案 3：Role-Based ===


100%|██████████| 100/100 [02:22<00:00,  1.42s/it]

模型准确率: 81.00%

📊 实验结果对比
方法                          准确率            相对提升
------------------------------------------------------------
Zero-Shot               73.00%           +0.0%
Chain-of-Thought        85.00%          +12.0%
Role-Based              81.00%           +8.0%
🏆 最佳方法: CoT (85.00%)


<font color="blue">You need to optimize the prompt to improve the performance (accuracy) of large language models (LLMs).</font>

## **Task 2: LLMS for AI Feedback**

The final stage of large language model training involves reinforcement learning through feedback. Such feedback can come from either  human experts or   AI. This feedback is used to learn a reward model, with data defined in a triplet form. This triplet comes from a question, two answers, and a choice by a human or AI on which answer is better.

The triplet consists of three elements: a  *question*, the  *chosen* answer, and the *rejected* answer. You are asked to use ChatGPT to provide the feedback, namely, choose the preferred one. Note that the feedback is highly biased by the order of placed answers, please shuffle the order of answers when using chatGPT for preference feedback.

In [ ]:
!wget  https://NLP-course-cuhksz.github.io/Assignments/Assignment1/task2/data/1.rlhf.jsonl

import json
with open('1.rlhf.jsonl') as f:
  data = [json.loads(l) for l in f]
data[0] # sample

--2025-03-06 22:50:09--  https://nlp-course-cuhksz.github.io/Assignments/Assignment1/task2/data/1.rlhf.jsonl
Resolving nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70333 (69K) [application/octet-stream]
Saving to: ‘1.rlhf.jsonl’

1.rlhf.jsonl        100%[===================>]  68.68K  --.-KB/s    in 0.01s   

2025-03-06 22:50:09 (5.36 MB/s) - ‘1.rlhf.jsonl’ saved [70333/70333]



{'Question': 'Human: What are some aspects of religious Islam that are compatible with western civilization?',
 'Answer1': "Assistant: I'm not sure what 'western civilization' means in this context, but if we assume it means the values of modern Western culture, it seems there are many facets of Islamic religious practice that are compatible with those values.  For example, a key principle of Islam is that there is one God who should be worshiped, which is consistent with the principle of religious tolerance, as well as the value of equality.  There are different branches of Islam, but many of them require",
 'Answer2': 'Assistant: Well in a basic sense it is possible for Muslims to live in the Western world and take part in Western civilization. In fact, many Muslims have been at the center of important cultural developments of the last centuries.',
 'Preference': 'Answer2'}

In [ ]:
data[0]

{'Question': 'Human: What are some aspects of religious Islam that are compatible with western civilization?',
 'Answer1': "Assistant: I'm not sure what 'western civilization' means in this context, but if we assume it means the values of modern Western culture, it seems there are many facets of Islamic religious practice that are compatible with those values.  For example, a key principle of Islam is that there is one God who should be worshiped, which is consistent with the principle of religious tolerance, as well as the value of equality.  There are different branches of Islam, but many of them require",
 'Answer2': 'Assistant: Well in a basic sense it is possible for Muslims to live in the Western world and take part in Western civilization. In fact, many Muslims have been at the center of important cultural developments of the last centuries.',
 'Preference': 'Answer2'}

In [ ]:
your_prompt = '''
[Question]:
{Question}

[Answer1]:
{Answer1}

[Answer2]:
{Answer2}

A good response should be relevant, accurate and helpful. Which is better, Answer1 or Answer2?
Do not explain your answer, just output 'Answer1' or 'Answer2'.
'''

def get_query(da):
  return your_prompt.format_map(da)

testdata = data[0]

print(get_query(testdata))

print(f'---- DeepSeek-Chat response ----')

print(chain.invoke(get_query(testdata)))


[Question]:
Human: What are some aspects of religious Islam that are compatible with western civilization?

[Answer1]:
Assistant: I'm not sure what 'western civilization' means in this context, but if we assume it means the values of modern Western culture, it seems there are many facets of Islamic religious practice that are compatible with those values.  For example, a key principle of Islam is that there is one God who should be worshiped, which is consistent with the principle of religious tolerance, as well as the value of equality.  There are different branches of Islam, but many of them require

[Answer2]:
Assistant: Well in a basic sense it is possible for Muslims to live in the Western world and take part in Western civilization. In fact, many Muslims have been at the center of important cultural developments of the last centuries.

A good response should be relevant, accurate and helpful. Which is better, Answer1 or Answer2?
Do not explain your answer, just output 'Answer1' 

In [ ]:
your_prompt = '''
[Question]:
{Question}

[Answer1]:
{Answer1}

[Answer2]:
{Answer2}

A good response should be relevant, accurate and helpful. Which is better, Answer1 or Answer2?
Do not explain your answer, just output 'Answer1' or 'Answer2'.
'''

import re
from tqdm import tqdm

correct_num = 0
total_num = 0
for da in tqdm(data[:10]):
  da['deepseek_ans'] =  chain.invoke(get_query(da))
  if da['deepseek_ans'] == da['Preference']:
    correct_num += 1
  total_num += 1
print(f"Model consistency rate with human: {correct_num/total_num:.2%}")

100%|██████████| 10/10 [01:35<00:00,  9.56s/it]

Model consistency rate with human: 50.00%


<font color="blue">You need to optimize the prompt to improve the performance (consistency rate) of large language models (LLMs).</font>